In [40]:
#The goal of this notebook is to analyse the data collected in Madrid that was used for the paper of Jelena 
#"Social experiments in the mesoscale: Humans playing a spatial Prisoner's Dilemma" PLoS ONE (in print)
#The data originally was given in a HEX format, meaning that the data has to be converted
#Extensive data cleaning and formatting ahead.


#In the experiment, volunteers played a 2x2 PD game with each of their eight neighbors (Moore neighborhood) taking only one action, either to cooperate (C) or to defect (D), the action being the same against all the opponents.

#Two experiment were conducted and the results were stored as series of Cs and Ds, separately for each of 169 users. These result were then compressed using hexadecimal notation. We coded any C as 1 and any D as 0. For instance, if someone played (only 12 rounds) CCDCCDCDDDCD will be, in binary notation, 110110100010 and in hexadecimal DA2. As the number of rounds are 47 and 58, therefore we added 1 and 2 zeros respectively at the end of the binary number, so as to make the number of binary digits a multiple of 4 and thus converted into hexadecimal straight away. 

#The other information needed to be stored are the positions of the players. This we stored as a position in the list. First hexadecimal number are the actions of the player on the position (1,1) in the lattice, the second is of the player on the position (1,2), the 14th number is of the player on the position (2,1) etc.

#Therefore, there are two files data_exp1.txt and data_exp2.txt with the actions of the players in the first experiment and second experiment respectively. Each contains the list of hexadecimal numbers with each number being the actions of the players and the positions in the list representing the position in the lattice. 
import pandas as pd
import hddm
import csv


/Users/eladiomontero/opt/anaconda3/envs/decTimesEnv/lib/python2.7/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


In [21]:
def hexToBin(hex_num):
    hexTobin_map = {
        "0":"0000",
        "1":"0001",
        "2":"0010",
        "3":"0011",
        "4":"0100",
        "5":"0101",
        "6":"0110",    
        "7":"0111",
        "8":"1000",
        "9":"1001",
        "A":"1010",
        "B":"1011",
        "C":"1100",
        "D":"1101",
        "E":"1110",
        "F":"1111",
    }
    return "".join(hexTobin_map[i] for i in hex_num)

In [31]:
def readFiles(url):
    for u in url:
        count = 0
        with open(u) as f:
            for line in f:
                hexToBin(line.strip())
                count += 1
        print count

In [33]:
readFiles(["./raw data/data_exp1.txt", "./raw data/data_exp2.txt", "./raw data/control/data_control.txt"])

169
169
169


In [50]:
hexToBin("C18414000000")

'110000011000010000010100000000000000000000000000'

In [41]:
reDict = {'D':0.,'C':1.}

def loadData():

	"""EXP1""" #with context
	#Load original file
	data = []
	with open('./raw data/times_exp1_rand.dat', 'r') as csvfile:
		csvreader = csv.reader(csvfile, delimiter=',')
		for row in csvreader:
			data.append((row[0],row[1],row[3],row[5],row[6].strip(),row[2]))
	#Create a file formatted for hddm
	with open('times_exp1_rand_hddm.csv', 'w') as outFile:
		outFile.write('userid,rt,response,round,context,strategy\n')
		for i in range(len(data)):
			d = data[i]
			action = reDict[d[1].strip()]
			roundN = int(d[2])
			if roundN > 1:
				context = contextNext #context is that of the previous round
			else:
				context = float('nan');
			contextNext = float(d[3])
			rt = float(d[0])/1000 #seconds
			if rt > 0 and rt< 30:
				outFile.write('%3d,%5.2f,%2.0f,%2d,%2.0f,%s\n'%(int(d[5]),rt,action,roundN,context,d[4]))
	#Load hddm format
	data1 = hddm.load_csv('times_exp1_rand_hddm.csv')
	data1 = hddm.utils.flip_errors(data1)

	"""CONTROL"""
	#Load original file
	data = []
	with open('./raw data/times_control_users.dat', 'r') as csvfile:
		csvreader = csv.reader(csvfile, delimiter=',')
		for row in csvreader:
			data.append((row[0],row[1],row[3],row[2]))
	#Create a file formatted for hddm
	with open('times_expC_rand_hddm.csv', 'w') as outFile:
		outFile.write('userid,rt,response,round\n')
		for d in data:
			rt = float(d[0])/1000; #seconds
			if rt > 0 and rt< 30:
				outFile.write('%3d,%8.6f,%5.3f,%2d\n'%(int(d[3]),rt,reDict[d[1].strip()],int(d[2])))
	#Load hddm format
	dataC = hddm.load_csv('times_expC_rand_hddm.csv')
	dataC = hddm.utils.flip_errors(dataC)

	"""EXP 2""" #with context
	#Load original file
	data = []
	with open('./raw data/times_exp2_rand.dat', 'r') as csvfile:
		csvreader = csv.reader(csvfile, delimiter=',')
		for row in csvreader:
			data.append((row[0],row[1],row[3],row[5],row[6].strip(),row[2]))
	#Create a file formatted for hddm
	with open('times_exp2_rand_hddm.csv', 'w') as outFile:
		outFile.write('userid,rt,response,round,context,strategy\n')
		for i in range(len(data)):
			d = data[i]
			action = reDict[d[1].strip()]
			roundN = int(d[2])
			if roundN > 1:
				context = contextNext #context is that of the previous round
			else:
				context = float('nan');
			contextNext = float(d[3])
			rt = float(d[0])/1000; #seconds
			if rt > 0 and rt< 30:
				outFile.write('%3d,%5.2f,%2.0f,%2d,%2.0f,%s\n'%(int(d[5]),rt,action,roundN,context,d[4]))
	#Load hddm format
	data2 = hddm.load_csv('times_exp2_rand_hddm.csv')
	data2 = hddm.utils.flip_errors(data2)

	"""PAIRWISE RECIPROCITY"""
	#Load original file
	data = []
	with open('./raw data/weakPD.dat', 'r') as csvfile:
		csvreader = csv.reader(csvfile, delimiter=' ')
		for row in csvreader:
			data.append((row[0],row[1],row[3],row[2]))
	#Create a file formatted for hddm
	with open('times_expW_rand_hddm.csv', 'w') as outFile:
		outFile.write('userid,rt,response,round\n')
		for d in data:
			rt = float(d[0])/1000; #seconds
			if rt > 0 and rt< 30:
				outFile.write('%2d,%8.6f,%5.3f,%2d\n'%(int(d[3]),rt,reDict[d[1].strip()],int(d[2])))
	#Load hddm format
	dataW = hddm.load_csv('times_expW_rand_hddm.csv')
	dataW = hddm.utils.flip_errors(dataW)

	return (data1,dataC,data2,dataW)

In [43]:
data1,dataC,data2,dataW = loadData()

In [47]:
data1.strategy.unique()

#Pure defectors: PD
#Pure cooperators: PC
#Moody conditional cooperators: MCC

array(['MCC', 'PD', 'PC'], dtype=object)

In [56]:
data1.context.unique()

array([nan,  4.,  3.,  1.,  2.,  0.,  6.,  5.,  7.])

In [51]:
hexToBin("C18414000000")

'110000011000010000010100000000000000000000000000'